# Lab 1.2 Using Boltz-1 model to predict protein structure

In [1]:
import os
import shutil
import numpy as np

## Helper functions

In [2]:

def preprare_directory(temp, delete_old=True):
    """
    Create a new directory and delete the old one if it exists
    :param temp: str: path to the directory
    :param delete_old: bool, whether to delete the old directory. Defaults to True.
    """
    if delete_old:  
        if os.path.exists(temp):
            # Remove the directory and all its contents
            shutil.rmtree(temp)
    # Recreate the directory
    os.makedirs(temp, exist_ok=True)

## Preparing the input file

[Boltz-1](https://github.com/jwohlwend/boltz/tree/main) is a state-of-the-art open-source model that predicts the 3D structure of proteins, RNA, DNA, and small molecules. It handles modified residues, covalent ligands and glycans, as well as condition the generation on pocket residues.

In this lab, we will primarily look at protein structure prediction. 2 exmaple fils are provided in the `notebooks/input` directory for monomer and multimer. Let's look at the multimer example first: 


```yaml
# notebooks/input/multimer.yaml
version: 1  # Optional, defaults to 1
sequences:
  - protein:
      id: A
      sequence: MAHHHHHHVAVDAVSFTLLQDQLQSVLDTLSEREAGVVRLRFGLTDGQPRTLDEIGQVYGVTRERIRQIESKTMSKLRHPSRSQVLRDYLDGSSGSGTPEERLLRAIFGEKA
  - protein:
      id: B
      sequence: MRYAFAAEATTCNAFWRNVDMTVTALYEVPLGVCTQDPDRWTTTPDDEAKTLCRACPRRWLCARDAVESAGAEGLWAGVVIPESGRARAFALGQLRSLAERNGYPVRDHRVSAQSA
```

To predict the structure, we will use the `boltz predict` command. 
Additional information about the command can be found using: 

```bash
! boltz predict --help
```

In [3]:
! boltz predict --help

Usage: boltz predict [OPTIONS] DATA

  Run predictions with Boltz-1.

Options:
  --out_dir PATH               The path where to save the predictions.
  --cache PATH                 The directory where to download the data and
                               model. Default is ~/.boltz.
  --checkpoint PATH            An optional checkpoint, will use the provided
                               Boltz-1 model by default.
  --devices INTEGER            The number of devices to use for prediction.
                               Default is 1.
  --accelerator [gpu|cpu|tpu]  The accelerator to use for prediction. Default
                               is gpu.
  --recycling_steps INTEGER    The number of recycling steps to use for
                               prediction. Default is 3.
  --sampling_steps INTEGER     The number of sampling steps to use for
                               prediction. Default is 200.
  --diffusion_samples INTEGER  The number of diffusion samples to use for
          

## Run predicdtion

### CLI

The command syntax looks like this: 

```text
boltz predict <input_yaml.yaml> --out_dir <output_dir> --devices 1 --output_format pdb --use_msa_server --diffusion_samples <number_of_predictions_to_generate>
```

- input file: this is the path to the input YAML file
- `out_dir`: Output directory to stre the result. 
- `devices`: number of GPUs. If you are providing a single YAML file, set to `1`. It distribute multiple runs across different GPUs
- `output_format`: Save the result in PDB format. 
- `use_msa_server`: Use the MSA server (Colab) to get the MSA. You can optionally provide your own MSA file, but for simplicity, we will use the server here. 
- `diffusion_samples`: Number of predictions to generate. 

Note: 
- **Some structures will require a large GPU memory**. E.g. `7WJ3` and `adalimumab` prediction work on g6e.8xlarge (L40S-48GB) GPU but will not work on GPUs with smaller memory such as g5 instances with A10-24GB or P3 instances with V100-16GB
- If memory is not sufficient, you will see the error such as: 
    ```bash
    ran out of memory, skipping batch
    ```
- If prediction is successful, you should see the following message: 
    ```bash
    Number of failed examples: 0
- The first time this command it is run, it will download the model weights and cache them, and can take a while (~6 min, depending on the network speed). 
- The second time it is run, it will read the cached weights, so it will be much faster, taking approximately a few minutes to complete


### Try it out

Run the keytruda example (VH/VL dimer) if GPU memory is 24GB: 


In [4]:
# ! boltz predict input/keytruda.yaml --out_dir output --devices 1 --output_format pdb --use_msa_server --num_workers 4

Run the [1JHL](https://www.rcsb.org/sequence/1JHL) example (VH/VL in complex with lysozme) if GPU memory is 48GB or larger: 

In [6]:
%%time
! boltz predict input/1JHL_min_48GB_GPU.yaml --out_dir output --devices 1 --output_format pdb --use_msa_server --diffusion_samples 1



Checking input data.
Running predictions for 1 structure
Processing input data.
  0%|                                                     | 0/1 [00:00<?, ?it/s]Generating MSA for input/1JHL_min_48GB_GPU.yaml with 3 protein entities.

COMPLETE: 100%|██████████████████████| 450/450 [elapsed: 00:01 remaining: 00:00]

PENDING:   0%|                             | 0/450 [elapsed: 00:00 remaining: ?]Sleeping for 8s. Reason: PENDING

RUNNING:   2%|▍                        | 8/450 [elapsed: 00:08 remaining: 08:00]Sleeping for 10s. Reason: RUNNING

RUNNING:   4%|▉                       | 18/450 [elapsed: 00:19 remaining: 07:34]Sleeping for 6s. Reason: RUNNING

RUNNING:   5%|█▎                      | 24/450 [elapsed: 00:25 remaining: 07:28]Sleeping for 8s. Reason: RUNNING

RUNNING:   7%|█▋                      | 32/450 [elapsed: 00:33 remaining: 07:18]Sleeping for 6s. Reason: RUNNING

RUNNING:   8%|██                      | 38/450 [elapsed: 00:40 remaining: 07:12]Sleeping for 8s. Reason: RUNNING


ON a L40S GPU, it roughly takes 3-4 mintues to predict the 1JHL complex

### Optional: try running in Python

First remove the old output directory if it exists: 

In [7]:
! rm -rf output/boltz_results_keytruda

In [2]:
import subprocess
input_yaml_path = "input/keytruda.yaml"
result_dir = "output"

# command to run the prediction
command = [
"boltz", 
"predict", 
input_yaml_path, 
"--out_dir", result_dir, 
"--devices", "1", 
"--output_format", "pdb", 
"--use_msa_server"
]

output = subprocess.run(command, capture_output=True, text=True)

In [4]:
print(output.stdout)

Checking input data.
Running predictions for 1 structure
Processing input data.
Generating MSA for input/keytruda.yaml with 2 protein entities.

Predicting: |                                                                                                                     | 0/? [00:00<?, ?it/s]
Predicting DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:29<00:00,  0.03it/s]Number of failed examples: 0

Predicting DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:29<00:00,  0.03it/s]



In [5]:
# make sure it is successful
assert "Number of failed examples: 0" in output.stdout


## Analyze the results

In the `notebooks/output/boltz_results_1JHL_min_48GB_GPU/predictions` directory, you should see: 
-  `1JHL_min_48GB_GPU_model_0.pdb` file: predicted structure
- `confidence_1JHL_min_48GB_GPU_model_0.json` file: confidence scores for the predicted structure
- `plddt_1JHL_min_48GB_GPU_model_0.npz` file: per-residue pLDDT confidence scores

Similarly to ESMFold, assume we want to visualize the predicted structure, coloring the residues based on the pLDDT scores. To do this we will first need to extract the pLDDT scores. 

In [8]:
# Load the .npz file that contains the pLDDT scores
npz_file_path = "output/boltz_results_1JHL_min_48GB_GPU/predictions/1JHL_min_48GB_GPU/plddt_1JHL_min_48GB_GPU_model_0.npz"
data = np.load(npz_file_path)
# extract the pLDDT scores
plddt_scores = data['plddt']

# how many scores are there?
print(plddt_scores.shape)

(353,)


> Note the length of the PLDDt scores = length of all 3 chains in the input YAML file. 

In [9]:
# take a look at the scores
plddt_scores

array([0.88314605, 0.9300585 , 0.9710183 , 0.984419  , 0.98278767,
       0.97389126, 0.9773581 , 0.97826964, 0.9398867 , 0.9398589 ,
       0.95806897, 0.969254  , 0.9765949 , 0.9703436 , 0.9620925 ,
       0.9777006 , 0.9731597 , 0.98440903, 0.9814968 , 0.9795893 ,
       0.98773855, 0.98705214, 0.9872477 , 0.9845588 , 0.965809  ,
       0.9747037 , 0.8487842 , 0.865317  , 0.89470685, 0.77851105,
       0.82245183, 0.8290827 , 0.9494724 , 0.9555825 , 0.9853915 ,
       0.97831804, 0.9792299 , 0.93544275, 0.92331576, 0.8237985 ,
       0.8198714 , 0.8544896 , 0.81063193, 0.88990986, 0.91380453,
       0.96261466, 0.98251   , 0.9669724 , 0.888028  , 0.8187067 ,
       0.9221771 , 0.93113816, 0.8692954 , 0.9530694 , 0.9428711 ,
       0.8985714 , 0.9384631 , 0.965975  , 0.9701984 , 0.96300673,
       0.97635573, 0.9854618 , 0.9760278 , 0.97947055, 0.9695035 ,
       0.96477354, 0.8872737 , 0.9136796 , 0.9630611 , 0.9790993 ,
       0.9829394 , 0.9853966 , 0.98831165, 0.98765475, 0.98623

If we take a look at the min and max values of the pLDDT scores, we can see that the scores are between 0 and 1. 

In [10]:

print(max(plddt_scores), min(plddt_scores))

0.98885936 0.57614946


For consistency with the ESMFold method, we will multiply the pLDDT scores by 100 to get the scores between 0 and 100.

## Visualizing the results with py3Dmol

From what we have seen above, to visaulize the protein, we need to match the pLDDT scores to the correct residues in the PDB file. 

Now open the `1JHL_min_48GB_GPU_model_0.pdb` file and take a look at the raw PDB file. 

Start counting columns from 0: 

- `column with index=0`: Label "ATOM" for most cases. If it is not "ATOM", ignore the line
- `column with index=1`: atom number, starting from 1
- `column with index=4`: this the chain ID: A, B, C, etc
- `column with index=5`: this is the residue number counting from 1 until end of chain A, then starting again from 1 for chain B, etc. 

This tells us our strategy could be: 
- read the PDB file line by line
- split each line by column
- Look at `column 1` (0 indexed), if it is not "ATOM", ignore the line
- Extract the residue number from `column 5` (0 indexed), and match it to the pLDDT score
- Style each atom in this residue based on the pLDDT score

Based on this, we will rewrite the `load_protein_boltz` function to visualize the results. 

In [23]:
import py3Dmol
def load_protein_boltz(pdb_file_path, plddt_file_path, width=800, height=600, color_by_chain=False):

    """
    Load a protein structure from a PDB file and display it using py3Dmol
    pdb_file_path: str, path to the PDB file
    plddt_file_path: str, path to the npz file containing the pLDDT scores
    width: int, width of the viewer in pixels
    height: int, height of the viewer in pixels
    color_by_chain: bool, whether to color the chains differently. Defaults to False.
    return: py3Dmol.view object
    """
    
    # load the pdb file
    with open(pdb_file_path) as ifile:
        pdb_data = "".join([x for x in ifile])

    # load the plddt scores
    scores = np.load(plddt_file_path)['plddt']
    
    view = py3Dmol.view(width=width, height=height)
    view.addModelsAsFrames(pdb_data)
    
    # if not coloring by chain, then color by pLDDT scores
    if not color_by_chain:
        print("color by pLDDT scores....")
        for line in pdb_data.split("\n"):

            # split each line by columns
            split = line.split()

            # not a valid line, ignore
            if len(split) == 0 or split[0] != "ATOM":
                continue

            # get residue id, pdb is 1-indexed, python is 0-indexed, therefore -1 to convert
            residue_idx = int(split[5]) - 1 

            # get the pLDDT score for the current residue, scale it to 0-100
            plddt_score = scores[residue_idx] * 100

            if plddt_score > 90:
                color = "blue"
            elif 70 <= plddt_score <= 90:
                color = "cyan"
            elif 50 <= plddt_score < 70:
                color = "yellow"
            else:
                color = "orange"
            
            # Atom serial numbers typically start from 1, similar to requirement of `view.setStyle`, hence idx should be used directly
            idx = int(split[1])
            
            # Style should be set per atom id
            view.setStyle({'model': -1, 'serial': idx}, {"cartoon": {'color': color}})

    # else color by chain
    else:
        print("color by chain....")
        chain_ids = set()
        # count the number of chains
        for line in pdb_data.split("\n"):
            split = line.split()

            # not a valid line, ignore
            if len(split) == 0 or split[0] != "ATOM":
                continue
            
            # split each line by columns
            chain_id = split[4]
            # add to the set
            chain_ids.add(chain_id)
        
        # palette
        palette = ["blue", "red", "yellow", "orange", "purple", "brown", "pink", "green", "cyan", "magenta"]
        for idx, cid in enumerate(chain_ids):
            adjusted_idx = idx % len(palette)
            view.setStyle({'chain': cid}, {'cartoon': {'color': palette[adjusted_idx]}})    
    view.zoomTo()
    return view


### visualize by chains

In [24]:
view = load_protein_boltz(
    pdb_file_path="output/boltz_results_1JHL_min_48GB_GPU/predictions/1JHL_min_48GB_GPU/1JHL_min_48GB_GPU_model_0.pdb", 
    plddt_file_path="output/boltz_results_1JHL_min_48GB_GPU/predictions/1JHL_min_48GB_GPU/plddt_1JHL_min_48GB_GPU_model_0.npz", 
    color_by_chain=True
)
view

color by chain....


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### visualize by pLDDT scores

In [25]:
# visualize by pLDDT scores
view = load_protein_boltz(
    pdb_file_path="output/boltz_results_1JHL_min_48GB_GPU/predictions/1JHL_min_48GB_GPU/1JHL_min_48GB_GPU_model_0.pdb", 
    plddt_file_path="output/boltz_results_1JHL_min_48GB_GPU/predictions/1JHL_min_48GB_GPU/plddt_1JHL_min_48GB_GPU_model_0.npz", 
    color_by_chain=False
)
view


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The coloring legend of the pLDDT score is same as the method used in AlphaFold database: 

![plddt](https://res.cloudinary.com/dpfqlyh21/image/upload/v1705026011/obsidian/izrfmiepbzpnzm2aoqwh.png)